# Onehot_A

In [1]:
# coding: utf-8

# filelist: 
#         train:40428967,  
#     minitrain:4042898,  
# miniminitrain:404291,  
#    test_click:4577464
#  

#import os
#os.chdir('/media/zhou/0004DD1700005FE8/AI/00/project_2/')
#os.chdir('E:/AI/00/project_2')


class flags(object):
    def __init__(self, file_name, onehot_cat):
        self.file_name = file_name
        self.onehot_cat = onehot_cat
        self.data_dir = '../data/project_2/data/{0}/'.format(self.onehot_cat)
        self.output_dir = '../data/project_2/output/{0}/'.format(self.onehot_cat)
        self.model_dir = '../data/project_2/models/{0}/'.format(self.onehot_cat)

class params(object):
    def __init__(self, onehot_name):
        self.threshold = 10
        self.chunksize = 1e3
        self.num_trees = 50
        self.deep = 15
        self.split = '='
        # ['A_cat', 'A_hour', 'A_xgb', 
        #  'B_cat', 'C_his']
        self.onehot_name = onehot_name
        self.lr_C = 1



In [2]:

import numpy as np
import pandas as pd
import time
import gc
import os
import scipy.sparse as ss
#from  sklearn.cross_validation  import  train_test_split 
import xgboost as xgb
from sklearn.metrics import accuracy_score,log_loss


In [3]:
def MergeNpz(output_path, file_name, data_begin, threshold=10,):
    """把生成的多个.npz合并成一个文件并保存"""
    #导入从0开始的文件, 为下面的合并做准备
    X_train = ss.load_npz(output_path+'{0}_X_more{1}_begin{2}.npz'.format(file_name, threshold, data_begin[0]),)
    y_train = ss.load_npz(output_path+'{0}_y_more{1}_begin{2}.npz'.format(file_name, threshold, data_begin[0]),)
    
    for begin in range(1,len(data_begin)): #循环读入文件
        #文件暂存
        X_train_tmp = ss.load_npz(output_path+'{0}_X_more{1}_begin{2}.npz'.format(
                                    file_name, threshold, data_begin[begin]),)
        y_train_tmp = ss.load_npz(output_path+'{0}_y_more{1}_begin{2}.npz'.format(
                                    file_name, threshold, data_begin[begin]),)
        X_train = ss.vstack((X_train, X_train_tmp))  #与原有 行稀疏矩阵 进行 行连接
        y_train = ss.hstack((y_train, y_train_tmp))  #与原有 列稀疏矩阵 进行 列连接
    
    print('total shape: ',X_train.shape, ' | ', y_train.shape)
    #保存最终连接完成的稀疏矩阵
    #ss.save_npz(output_path+'{0}_X_more{1}'.format(file_name, threshold), X_train)
    #ss.save_npz(output_path+'{0}_y_more{1}'.format(file_name, threshold), y_train)
    #print('saved in {0}'.format(output_path))
    return X_train, y_train


In [4]:
def LogLoss(bst_train, xgtrain):
    train_preds = bst_train.predict(xgtrain)
    train_predictions = [round(value) for value in train_preds]
    y_train = xgtrain.get_label()
    train_accuracy = accuracy_score(y_train, train_predictions)
    print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
    train_log_loss = log_loss(y_train, train_preds)
    print ("Train log_loss: " , train_log_loss)

In [5]:
gc.collect()

532

In [6]:
totalsize = {'minitrain':4042898, 'test_click':4577464}
file_size = totalsize[FLAGS.file_name]  #总的数据量
block_size = 100000  #数据块大小

#定义参数
data_path = FLAGS.data_dir  # + '../data/project_2/'
file_name = FLAGS.file_name
chunksize = 1000
threshold = 10
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

data_begins = [i for i in range(0,file_size,block_size)]



# 读入数据并转为xgb.DMatrix

In [32]:
#导入数据
threshold = 'A_cat'
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, threshold), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, threshold), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [33]:
xgtrain = xgb.DMatrix(X_train, label = y_train,)
xgtest = xgtrain.slice(range(3500000, 4042898))
xgtrain = xgtrain.slice(range(3500000))

In [34]:
xgtrain.num_row(), xgtest.num_row(),

(3500000, 542898)

In [35]:
print(X_train.shape, y_train.shape)
#print(X_val.shape, y_val.shape)

(4042898, 645142) (4042898,)


X_test = ss.load_npz(data_path+'minitest_X_more10.npz', )
y_test = ss.load_npz(data_path+'minitest_y_more10.npz', )
y_test = y_test.toarray().astype(np.float32)[0]

In [36]:
watchlist=[(xgtrain,'train'), (xgtest,'eval')]

# A_cat

In [ ]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [41]:
#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [42]:
xgtrain = xgb.DMatrix(X_train, label = y_train,)
xgtest = xgtrain.slice(range(3500000, 4042898))
xgtrain = xgtrain.slice(range(3500000))

xgtrain.num_row(), xgtest.num_row(),

watchlist=[(xgtrain,'train'), (xgtest,'eval')]

In [45]:
#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=12, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)


num_trees = 40 #树的数量##################

#调用cv函数
bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
#bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))
bst_train[-10:]

training . . . 
cost time:249


,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
30,0.399262,0.000037,0.397840,0.000183
31,0.399185,0.000025,0.397733,0.000183
32,0.399117,0.000042,0.397650,0.000189
33,0.399002,0.000007,0.397494,0.000140
34,0.398929,0.000035,0.397403,0.000121
35,0.398863,0.000046,0.397331,0.000105
36,0.398783,0.000105,0.397233,0.000068
37,0.398721,0.000109,0.397144,0.000066
38,0.398659,0.000114,0.397053,0.000048
39,0.398611,0.000101,0.396990,0.000072


In [46]:
bst_train = xgb.train(param, xgtrain, num_trees, evals=watchlist) 

[0]	train-logloss:0.451957	eval-logloss:0.450707
[1]	train-logloss:0.419697	eval-logloss:0.419285
[2]	train-logloss:0.412605	eval-logloss:0.413671
[3]	train-logloss:0.40737	eval-logloss:0.40987
[4]	train-logloss:0.405732	eval-logloss:0.408914
[5]	train-logloss:0.404834	eval-logloss:0.408147
[6]	train-logloss:0.404127	eval-logloss:0.407441
[7]	train-logloss:0.403617	eval-logloss:0.406664
[8]	train-logloss:0.403026	eval-logloss:0.406478
[9]	train-logloss:0.402047	eval-logloss:0.405182
[10]	train-logloss:0.401572	eval-logloss:0.404795
[11]	train-logloss:0.401107	eval-logloss:0.404507
[12]	train-logloss:0.400748	eval-logloss:0.404357
[13]	train-logloss:0.400519	eval-logloss:0.404257
[14]	train-logloss:0.400247	eval-logloss:0.404105
[15]	train-logloss:0.400011	eval-logloss:0.404001
[16]	train-logloss:0.399758	eval-logloss:0.403744
[17]	train-logloss:0.399588	eval-logloss:0.404201
[18]	train-logloss:0.399253	eval-logloss:0.403879
[19]	train-logloss:0.399077	eval-logloss:0.403766
[20]	train-l

# A_hour

In [47]:
#导入数据
threshold = 'A_hour'
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, threshold), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, threshold), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [48]:
xgtrain = xgb.DMatrix(X_train, label = y_train,)
xgtest = xgtrain.slice(range(3500000, 4042898))
xgtrain = xgtrain.slice(range(3500000))

In [49]:
xgtrain.num_row(), xgtest.num_row(),

(3500000, 542898)

In [50]:
watchlist=[(xgtrain,'train'), (xgtest,'eval')]

In [51]:
#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=12, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)


num_trees = 40 #树的数量##################

#调用cv函数
bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
#bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))
bst_train[-10:]

training . . . 
cost time:79


,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
30,0.455346,0.000048,0.455272,0.000024
31,0.455350,0.000048,0.455272,0.000024
32,0.455352,0.000051,0.455272,0.000024
33,0.455351,0.000050,0.455273,0.000024
34,0.455352,0.000050,0.455272,0.000024
35,0.455351,0.000050,0.455272,0.000024
36,0.455352,0.000049,0.455272,0.000024
37,0.455350,0.000050,0.455272,0.000024
38,0.455350,0.000049,0.455273,0.000024
39,0.455349,0.000048,0.455272,0.000024


In [52]:
bst_train = xgb.train(param, xgtrain, num_trees, evals=watchlist) 

[0]	train-logloss:0.477905	eval-logloss:0.479359
[1]	train-logloss:0.457336	eval-logloss:0.456814
[2]	train-logloss:0.455432	eval-logloss:0.454008
[3]	train-logloss:0.455307	eval-logloss:0.453519
[4]	train-logloss:0.455293	eval-logloss:0.453483
[5]	train-logloss:0.455287	eval-logloss:0.453407
[6]	train-logloss:0.455285	eval-logloss:0.453444
[7]	train-logloss:0.455284	eval-logloss:0.453395
[8]	train-logloss:0.455284	eval-logloss:0.453419
[9]	train-logloss:0.455284	eval-logloss:0.453413
[10]	train-logloss:0.455284	eval-logloss:0.453445
[11]	train-logloss:0.455284	eval-logloss:0.453435
[12]	train-logloss:0.455284	eval-logloss:0.453433
[13]	train-logloss:0.455284	eval-logloss:0.453443
[14]	train-logloss:0.455284	eval-logloss:0.453465
[15]	train-logloss:0.455284	eval-logloss:0.453423
[16]	train-logloss:0.455284	eval-logloss:0.453413
[17]	train-logloss:0.455284	eval-logloss:0.453421
[18]	train-logloss:0.455284	eval-logloss:0.453466
[19]	train-logloss:0.455284	eval-logloss:0.453418
[20]	train

# A_xgb

In [37]:
#导入数据
threshold = 'A_xgb'
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, threshold), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, threshold), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

Load Data


In [38]:
xgtrain = xgb.DMatrix(X_train, label = y_train,)
xgtest = xgtrain.slice(range(3500000, 4042898))
xgtrain = xgtrain.slice(range(3500000))

In [39]:
xgtrain.num_row(), xgtest.num_row(),

(3500000, 542898)

In [40]:
watchlist=[(xgtrain,'train'), (xgtest,'eval')]

In [20]:
#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=12, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)


num_trees = 40 #树的数量##################

#调用cv函数
bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
#bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))
bst_train

training . . . 
cost time:241


,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.454386,0.002213,0.454181,0.002200
1,0.420969,0.001043,0.420669,0.001060
2,0.412808,0.000596,0.412364,0.000644
3,0.409807,0.000742,0.409321,0.000798
4,0.407287,0.000494,0.406712,0.000568
5,0.406366,0.000514,0.405744,0.000558
6,0.405396,0.000553,0.404741,0.000590
7,0.404684,0.000429,0.403983,0.000465
8,0.404063,0.000321,0.403265,0.000375
9,0.403475,0.000315,0.402651,0.000390


bst_train = xgb.train(param, xgtrain, num_trees, evals=watchlist) 

# 正确率与模型保存

In [6]:
from sklearn.metrics import accuracy_score,log_loss

In [7]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=deep, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)

#调用cv函数
#bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))

train_preds = bst_train.predict(xgtrain)
train_predictions = [round(value) for value in train_preds]
y_train = xgtrain.get_label()
train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)

bst_train.save_model(model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))
print('saved in: ', model_path)

Load Data
training . . . 
cost time:116
Train Accuary: 83.47%
Train log_loss:  0.397309582592
saved in:  ../data/project_2/models/Onehot_A/


In [8]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_hour')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=deep, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)

#调用cv函数
#bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))

train_preds = bst_train.predict(xgtrain)
train_predictions = [round(value) for value in train_preds]
y_train = xgtrain.get_label()
train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)

bst_train.save_model(model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))
print('saved in: ', model_path)

Load Data
training . . . 
cost time:33
Train Accuary: 83.00%
Train log_loss:  0.454812237607
saved in:  ../data/project_2/models/Onehot_A/


In [9]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_xgb')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=deep, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)

#调用cv函数
#bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))

train_preds = bst_train.predict(xgtrain)
train_predictions = [round(value) for value in train_preds]
y_train = xgtrain.get_label()
train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)

bst_train.save_model(model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))
print('saved in: ', model_path)

Load Data


FileNotFoundError: [Errno 2] No such file or directory: '../data/project_2/output/Onehot_A/minitrain_X_A_xgb_all.npz'

In [10]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_his')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=deep, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)

#调用cv函数
#bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))

train_preds = bst_train.predict(xgtrain)
train_predictions = [round(value) for value in train_preds]
y_train = xgtrain.get_label()
train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)

bst_train.save_model(model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))
print('saved in: ', model_path)

Load Data
training . . . 
cost time:18
Train Accuary: 83.02%
Train log_loss:  0.452250324549
saved in:  ../data/project_2/models/Onehot_A/


In [11]:
FLAGS = flags('minitrain', 'Onehot_B')
PARAMS = params('B_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

param = dict(
        learning_rate =0.8, 
        booster='gbtree',
        n_estimators=100,  
        max_depth=deep, 
        min_child_weight=50,
        gamma=0,
        subsample=0.9,
        colsample_bytree=0.8,
        colsample_bylevel=0.7,
        objective= 'binary:logistic' ,
        eta=0.4,
        silent=0,
        eval_metric='logloss',
        seed=3)

#调用cv函数
#bst_train = xgb.cv(param, xgtrain, num_trees, nfold=3, stratified=True)
bst_train = xgb.train(param, xgtrain, num_trees, ) 
#new_feature = bst_train.predict(xgtrain, pred_leaf=True)
print('cost time:{0}'.format(int(time.time() - start_time)))

train_preds = bst_train.predict(xgtrain)
train_predictions = [round(value) for value in train_preds]
y_train = xgtrain.get_label()
train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)

bst_train.save_model(model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))
print('saved in: ', model_path)

Load Data
training . . . 
cost time:38
Train Accuary: 83.01%
Train log_loss:  0.455020206432
saved in:  ../data/project_2/models/Onehot_B/


In [70]:
bst_train = xgb.Booster(model_file= model_path + 'xgboost.model')